# Imports

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from albumentations import (Compose, HorizontalFlip, VerticalFlip, RandomRotate90, Normalize, Resize)
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanIoU

# Preprocessing

In [2]:
def load_data(data_dir):
    images = []
    masks = []
    for patient_folder in os.listdir(data_dir):
        folder_path = os.path.join(data_dir, patient_folder)
        if os.path.isdir(folder_path):
            for filename in os.listdir(folder_path):
                if filename.endswith('.tif'):  # Only process .tif files
                    img_path = os.path.join(folder_path, filename)
                    if '_mask.tif' in filename:
                        mask_path = img_path  # This is a mask file
                    else:
                        mask_path = os.path.join(folder_path, filename.replace('.tif', '_mask.tif'))
                    
                    # Load image and mask
                    image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

                    # Check for missing masks
                    if mask is not None and image is not None:
                        images.append(image)
                        masks.append(mask)

    return np.array(images), np.array(masks)

In [3]:
def apply_clahe(images):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    return np.array([clahe.apply(image) for image in images])

In [14]:
def get_augmentation():
    return Compose([
        Resize(160, 160),
        HorizontalFlip(),
        VerticalFlip(),
        RandomRotate90(),
        Normalize(mean=0.5, std=0.5)
    ])

In [5]:
data_dir = './Data'  # Change this to your dataset path
images, masks = load_data(data_dir)
images_clahe = apply_clahe(images)

In [6]:
len(images_clahe)
len(masks)

7858

In [15]:
# Assuming images_clahe and masks are your original datasets
num_images = min(len(images_clahe), 3000)

# Randomly select indices
random_indices = np.random.choice(len(images_clahe), num_images, replace=False)

# Create subsets for images and masks
X_random = images_clahe[random_indices]
y_random = masks[random_indices]

# Apply augmentation to the randomly selected images
augmentation = get_augmentation()

# Resize and augment images
X_augmented = np.array([augmentation(image=image)['image'] for image in X_random])
y_augmented = np.array([augmentation(image=image)['image'] for image in y_random])


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_augmented, y_augmented, test_size=0.2, random_state=42)

In [17]:
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)
y_train = np.expand_dims(y_train, axis=-1)
y_test = np.expand_dims(y_test, axis=-1)

# Model Training

In [18]:
def conv_block(input_tensor, num_filters):
    """
    Convolutional block consisting of two convolutional layers followed by batch normalization and ReLU activation.
    """
    x = layers.Conv2D(num_filters, 3, padding='same')(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(num_filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    return x

def nested_unet(input_shape):
    """
    Build the Nested U-Net model.
    """
    inputs = layers.Input(input_shape)

    # Level 1
    conv1 = conv_block(inputs, 64)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    # Level 2
    conv2 = conv_block(pool1, 128)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    # Level 3
    conv3 = conv_block(pool2, 256)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    # Level 4
    conv4 = conv_block(pool3, 512)
    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv4)

    # Bottleneck
    bottleneck = conv_block(pool4, 1024)

    # Nested skip connections (U-Net++ architecture)
    # Level 4
    up4 = layers.UpSampling2D(size=(2, 2))(bottleneck)  # Up-sample to match conv4
    up4 = layers.Conv2D(512, (1, 1), padding='same')(up4)  # Adjust channels
    nested4 = layers.Concatenate()([conv4, up4])  # Concatenate with the skip connection
    conv4_1 = conv_block(nested4, 512)

    # Level 3
    up3 = layers.UpSampling2D(size=(2, 2))(conv4_1)  # Up-sample to match conv3
    up3 = layers.Conv2D(256, (1, 1), padding='same')(up3)  # Adjust channels
    nested3 = layers.Concatenate()([conv3, up3])  # Concatenate with the skip connection
    conv3_1 = conv_block(nested3, 256)

    # Level 2
    up2 = layers.UpSampling2D(size=(2, 2))(conv3_1)  # Up-sample to match conv2
    up2 = layers.Conv2D(128, (1, 1), padding='same')(up2)  # Adjust channels
    nested2 = layers.Concatenate()([conv2, up2])  # Concatenate with the skip connection
    conv2_1 = conv_block(nested2, 128)

    # Level 1
    up1 = layers.UpSampling2D(size=(2, 2))(conv2_1)  # Up-sample to match conv1
    up1 = layers.Conv2D(64, (1, 1), padding='same')(up1)  # Adjust channels
    nested1 = layers.Concatenate()([conv1, up1])  # Concatenate with the skip connection
    conv1_1 = conv_block(nested1, 64)

    # Output layer
    outputs = layers.Conv2D(1, 1, activation='sigmoid')(conv1_1)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model

# Define input shape
input_shape = (160, 160, 1)  # Assuming grayscale images
nested_unet_model = nested_unet(input_shape)

# Compile the model
nested_unet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
nested_unet_model.summary()  # Print the model summary


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 160, 160, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_73 (Conv2D)             (None, 160, 160, 64  640         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_62 (BatchN  (None, 160, 160, 64  256        ['conv2d_73[0][0]']              
 ormalization)                  )                                                           

In [11]:
def conv_block(input_tensor, num_filters):
    """Convolutional block with two convolutional layers, batch normalization, and ReLU activation."""
    x = layers.Conv2D(num_filters, 3, padding='same')(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(num_filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    return x

def attention_block(skip_tensor, input_tensor, num_filters):
    """Attention block that computes attention weights based on the skip connections and the input tensor."""
    # Compute the attention mechanism
    g = layers.Conv2D(num_filters, 1, padding='same')(input_tensor)  # 1x1 convolution for gating
    g = layers.BatchNormalization()(g)
    g = layers.Activation('relu')(g)

    # Skip connection
    x = layers.Conv2D(num_filters, 1, padding='same')(skip_tensor)  # 1x1 convolution for skip connection
    x = layers.BatchNormalization()(x)

    # Element-wise addition and sigmoid activation
    x = layers.Add()([g, x])
    x = layers.Activation('sigmoid')(x)

    # Multiply the attention weights with the input tensor
    return layers.Multiply()([input_tensor, x])

def attention_unet(input_shape):
    """Build the Attention U-Net model."""
    inputs = layers.Input(input_shape)

    # Encoder path
    conv1 = conv_block(inputs, 64)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = conv_block(pool1, 128)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = conv_block(pool2, 256)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = conv_block(pool3, 512)
    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv4)

    # Bottleneck
    bottleneck = conv_block(pool4, 1024)

    # Decoder path
    # Level 4
    up4 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(bottleneck)
    att4 = attention_block(conv4, up4, 512)  # Apply attention
    conv4_1 = conv_block(att4, 512)

    # Level 3
    up3 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv4_1)
    att3 = attention_block(conv3, up3, 256)  # Apply attention
    conv3_1 = conv_block(att3, 256)

    # Level 2
    up2 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv3_1)
    att2 = attention_block(conv2, up2, 128)  # Apply attention
    conv2_1 = conv_block(att2, 128)

    # Level 1
    up1 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv2_1)
    att1 = attention_block(conv1, up1, 64)  # Apply attention
    conv1_1 = conv_block(att1, 64)

    # Output layer
    outputs = layers.Conv2D(1, 1, activation='sigmoid')(conv1_1)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model

# Define input shape
input_shape = (160, 160, 1)  # Assuming grayscale images
attention_unet_model = attention_unet(input_shape)

# Compile the model
attention_unet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
attention_unet_model.summary()  # Print the model summary


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 160, 160, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_23 (Conv2D)             (None, 160, 160, 64  640         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_18 (BatchN  (None, 160, 160, 64  256        ['conv2d_23[0][0]']              
 ormalization)                  )                                                           

In [19]:
# Define DICE Score
def dice_score(y_true, y_pred, smooth=1e-6):
    intersection = np.sum(y_true * y_pred)
    return (2. * intersection + smooth) / (np.sum(y_true) + np.sum(y_pred) + smooth)

In [20]:
input_shape = (160, 160, 1)

# Nested U-Net
nested_model = nested_unet(input_shape)
nested_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=[MeanIoU(num_classes=2)])
nested_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30)

Epoch 1/30


ResourceExhaustedError: Graph execution error:

Detected at node 'model_4/conv2d_96/Conv2D' defined at (most recent call last):
    File "c:\Users\mayan\.conda\envs\tf210\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\mayan\.conda\envs\tf210\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\mayan\.conda\envs\tf210\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\mayan\.conda\envs\tf210\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "c:\Users\mayan\.conda\envs\tf210\lib\site-packages\ipykernel\kernelapp.py", line 736, in start
      self.io_loop.start()
    File "c:\Users\mayan\.conda\envs\tf210\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\mayan\.conda\envs\tf210\lib\asyncio\base_ev

In [ ]:
# Attention U-Net
attention_model = attention_unet(input_shape)
attention_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=[MeanIoU(num_classes=2)])
attention_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30)